In [ ]:
import config as k
import pandas as pd
import ccxt
import pandas_ta as ta
import decimal
import time

binance = ccxt.binance({'enableRateLimit':True,
                        'apiKey': k.binancekey,
                        'secret': k.binancesecret,
                        'options':{
                            'defaultType': 'future',
                        }
})


bal = binance.fetch_positions(symbols=['XRPUSDT'])

In [ ]:
# symbol = 'XRPUSDT'
# livro_ofertas = binance.fetch_order_book(symbol)
# bid = decimal.Decimal(livro_ofertas['bids'][0][0])
# ask = decimal.Decimal(livro_ofertas['asks'][0][0])
# print(bid, ask)

def livro_ofertas(symbol):
    livro_ofertas = binance.fetch_order_book(symbol)
    bid = decimal.Decimal(livro_ofertas['bids'][0][0])
    ask = decimal.Decimal(livro_ofertas['asks'][0][0])
    return bid, ask


In [ ]:
def posicoes_abertas(symbol):
    lado = []
    tamanho = []
    preco_entrada = []
    notional = []
    percentage = []
    pnl = []
    bal = binance.fetch_positions(symbols=[symbol])
    for i in bal:
        lado = i['side']
        tamanho = i['info']['positionAmt'].replace('-','')
        notional = i['notional']
        preco_entrada = i['entryPrice']
        percentage = i['percentage']
        pnl = i['info']['unRealizedProfit']

    if lado == 'long':
        pos_aberta = True
    elif lado == 'short':
        pos_aberta = True
    else:
        pos_aberta = False          

    return lado, tamanho, preco_entrada, pos_aberta, notional, percentage, pnl

posicoes_abertas('XRPUSDT')



In [ ]:
# symbol = 'XRPUSDT'

def encerra_posicao(symbol):
    pos_aberta = posicoes_abertas(symbol)[3]
    while pos_aberta == True:
        lado = posicoes_abertas(symbol)[0]
        tamanho = posicoes_abertas(symbol)[1]

        if lado == 'long':
            binance.cancel_all_orders(symbol)
            bid, ask = livro_ofertas(symbol)
            ask = binance.price_to_precision(symbol, ask)
            binance.create_order(symbol, side='sell', type='LIMIT', price=ask, amount=tamanho, params= {'hedged':'true'})
            print(f'Vendendo posição long de {tamanho} moedas de {symbol}')
            # msg = 'Vendendo posiçao...'
            # telegram
            time.sleep(20)

        elif lado == 'short':
            binance.cancel_all_orders(symbol)
            bid, ask = livro_ofertas(symbol)
            bid = binance.price_to_precision(symbol, bid)
            binance.create_order(symbol, side='buy', type='LIMIT', price=bid, amount=tamanho, params= {'hedged':'true'})
            print(f'Comprando posição short de {tamanho} moedas de {symbol}')
            time.sleep(20)
        else:
            print('Impossível encerrar a posição!')

        pos_aberta = posicoes_abertas(symbol=symbol)[3]


In [ ]:
loss = -10
target = 20

def fecha_pnl(symbol, loss, target):
    percent = posicoes_abertas(symbol=symbol)[5]
    pnl = posicoes_abertas(symbol=symbol)[6]
    if percent:
        if percent <= loss:
            print(f'Encerrando posição por loss! {pnl}')
            encerra_posicao(symbol)
            #telegram
            # time.sleep(3000)
        elif percent >= gain:
            print(f'Encerra posiçao por gain! {pnl}')
            #telegram


In [ ]:
symbol = 'BTCUSDT'
timeframe = '5m'
bars = binance.fetch_ohlcv(symbol=symbol, timeframe=timeframe, limit=100)

In [ ]:
df_candles = pd.DataFrame(bars, columns=['time', 'abertura', 'max', 'min', 'fechamento', 'volume'])
df_candles['time'] = pd.to_datetime(df_candles['time'], unit='ms', utc=True).dt.tz_convert('America/Sao_Paulo')


In [ ]:
df_candles

In [ ]:
def posicao_max(symbol, max_pos):
    pos = posicoes_abertas(symbol)[1]
    if isinstance(pos, list):
        max_posicao = False
    elif float(pos) >= max_pos:
        max_posicao = True
    else:
        max_posicao = False
    return max_posicao

# posicao_max('XRPUSDT', 4000)

In [ ]:
def ultima_ordem_aberta(symbol):
    order = []
    order = binance.fetch_orders(symbol)[-1]['status']
    if order == 'open':
        open_order = True
    else:
        open_order = False
    return open_order


In [3]:
import config as k
import pandas as pd
import ccxt
import pandas_ta as ta
import decimal
import time
from ta.momentum import RSIIndicator

binance = ccxt.binance({'enableRateLimit':True,
                        'apiKey': k.binancekey,
                        'secret': k.binancesecret,
                        'options':{
                            'defaultType': 'future',
                        }
})

symbol = 'ETHUSDT'
timeframe = '15m'
bars = binance.fetch_ohlcv(symbol=symbol, timeframe=timeframe, limit=500)
df_candles = pd.DataFrame(bars, columns=['time', 'abertura', 'max', 'min', 'fechamento', 'volume'])
df_candles['time'] = pd.to_datetime(df_candles['time'], unit='ms', utc=True).map(lambda x: x.tz_convert('America/Sao_Paulo'))


In [4]:
df_candles['suporte'] = df_candles['min'].rolling(window=10).min()
df_candles['resistencia'] = df_candles['max'].rolling(window=10).max()

In [5]:
def calc_sup_rest(df, tamanho=30):
    df['suporte'] = df['min'].rolling(window=10).min()
    df['resistencia'] = df['max'].rolling(window=10).max()

    return df.iloc[-2]['suporte'], df.iloc[-2]['resistencia']

sup, res = calc_sup_rest(df_candles)

In [14]:
symbol = 'ETHUSDT'
timeframe = '15m'
bars = binance.fetch_ohlcv(symbol=symbol, timeframe=timeframe, limit=50)
df_candles = pd.DataFrame(bars, columns=['time', 'abertura', 'max', 'min', 'fechamento', 'volume'])
df_candles['time'] = pd.to_datetime(df_candles['time'], unit='ms', utc=True).map(lambda x: x.tz_convert('America/Sao_Paulo'))

#definir stop loss e gain
stop_loss = 0.01
take_profit = 0.04

#moedas para operar e possicao MAXIMA!
alt_coins = {'LINK/USDT': 20, 'CHZ/USDT':3000, '1000SHIB/USDT':1500, 'TIA/USDT':50}

#Aplicar funcao sup e resistencia
suporte, resistencia = calc_sup_rest(df_candles)

#preço do eth
price = binance.fetch_trades(symbol)[-1]['price']

if price > 0:
    coinData = {}

    for coin in alt_coins:
        altcoins_price = binance.fetch_trades(coin)[-1]['price']
        coinData[coin] = abs((altcoins_price - price) / price) * 100
        print(coinData[coin])


99.51161269119558
99.9976545469347
99.99924007200079
99.83171170736561


In [6]:
sup, res

(3267.28, 3324.21)

In [ ]:
smas = [5, 10, 50, 200]

for sma in smas:
    df_candles[f'SMS_{sma}'] = df_candles['fechamento'].rolling(sma).mean()

df_candles['SMA_20'] = ta.sma(close=df_candles.fechamento, length=20)
df_candles['EMA_20'] = ta.ema(close=df_candles.fechamento, length=20)

In [ ]:
rsi = RSIIndicator(df_candles['fechamento'])
df_candles['RSI'] = rsi.rsi()

In [ ]:
bollinger_bands = ta.bbands(df_candles.fechamento, length=20, std=2)

In [ ]:
bollinger_bands = bollinger_bands.iloc[:,[0,1,2]]
bollinger_bands.columns = ['BBL', 'BBM', 'BBU']

df_candles = pd.concat([df_candles, bollinger_bands], axis=1)

In [ ]:
macd = df_candles.ta.macd(close='fechamento', fast=12, slow=26, signal=9, append=True)
# df_candles = pd.concat([df_candles, macd], axis=1)

In [ ]:
df_candles['preco_ponderado'] = df_candles['fechamento'] * df_candles['volume']
df_candles['VWAP'] = df_candles['preco_ponderado'].sum() / df_candles['volume'].sum()

In [ ]:
print(df_candles)